In [3]:
import dask.dataframe as dd

df = dd.read_parquet("hf://datasets/OpenCo7/UpVoteWeb/data/*.parquet")

/home/nskandera/myenv/lib/python3.9/site-packages/dask/dataframe/io/parquet/core.py:1247: FutureWarning: Could not find pyarrow; falling back to fastparquet, which is deprecated and will be removed in a future release.
  warnings.warn(


In [7]:
df.head(5)


,id,parent_id,post_id,text,url,date,author,subreddit,score,token_count,kind,language,language_score,media_urls
0,kfrxfcr,t3_18vki9b,18vki9b,"""devsisters is so generous guys"" the generosity:",https://www.reddit.com/r/CookieRunKingdoms/com...,2024-01-01T00:59:58,Rotelle,CookieRunKingdoms,35,13,comment,en,0.8768,[]
1,kfrxfdj,t3_18vlhgc,18vlhgc,Omar Epps waiting in the wings to star in the ...,https://www.reddit.com/r/nfl/comments/18vlhgc/...,2024-01-01T00:59:58,SQRTLURFACE,nfl,253,19,comment,en,0.8109,[]
2,kfrxfde,t3_18vkkxu,18vkkxu,May 1899 be a better year for us all than 1899...,https://www.reddit.com/r/fallenlondon/comments...,2024-01-01T00:59:58,TyrconnellFL,fallenlondon,39,15,comment,en,0.9628,[]
3,kfrxfda,t3_18vkqmn,18vkqmn,Just clean it up a bit. We did almost the same...,https://www.reddit.com/r/Renovations/comments/...,2024-01-01T00:59:58,RampDog1,Renovations,1,32,comment,en,0.9488,[]
4,kfrxfdl,t1_kfrwiiz,18vl745,Absolutely pleasant little guys. Just a discla...,https://www.reddit.com/r/PokeMedia/comments/18...,2024-01-01T00:59:58,GameSpection,PokeMedia,7,46,comment,en,0.9757,[]
